In [1]:
import pandas as pd
import numpy as np 
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
'''Meaning behind project is to see if you can predict success in 
big leagues through minor league success (min 100 ABs)]'''
####Took seasons 2014-2018
szn_2014 = pd.read_csv('2014_milb.csv')
szn_2015 = pd.read_csv('2015_milb.csv')
szn_2016 = pd.read_csv('2016_milb.csv')
szn_2017 = pd.read_csv('2017_milb.csv')
szn_2018 = pd.read_csv('2018_milb.csv')
pros = pd.read_csv('pros_2019.csv')

In [3]:
df = pd.concat([szn_2014,szn_2015,szn_2016,szn_2017,szn_2018])
data = df.drop(['Rk','Lg','Tm','Aff','G','PA','AB','GDP','Lev'],axis=1)
data = data.sort_values('OPS',ascending=False)
data

,Name,Age,OPS,R,H,2B,3B,HR,RBI,SB,...,BB,SO,BA,OBP,SLG,TB,HBP,SH,SF,IBB
0,Malcom Nunez,17,1.272,44,68,16,2,13,59,3,...,26,29,0.415,0.497,0.774,127,5,0,4,3
1,Juan Soto,19,1.218,33,55,10,4,14,52,3,...,29,28,0.362,0.462,0.757,115,0,0,1,1
0,Michael Conforto,23,1.209,30,54,8,2,9,28,2,...,13,18,0.422,0.483,0.727,93,2,0,0,0
1,Ryan Schimpf,28,1.160,36,59,17,0,15,48,0,...,21,33,0.355,0.432,0.729,121,2,0,1,1
0,Jorge Soler,22,1.132,42,68,23,2,15,57,0,...,33,48,0.340,0.432,0.700,140,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Edgar Rondon,20,0.902,36,56,8,7,1,27,12,...,29,37,0.322,0.437,0.466,81,8,1,2,3
97,Eloy Jimenez,19,0.901,65,142,40,3,14,81,8,...,25,94,0.329,0.369,0.532,230,4,0,3,3
99,Amalani Fukofuka,19,0.901,53,95,18,9,3,38,10,...,26,70,0.339,0.401,0.500,140,4,1,2,1
99,Jesus Sanchez,18,0.900,33,70,10,8,7,39,2,...,9,43,0.329,0.351,0.549,117,0,1,3,0


In [4]:
pros = pros.dropna()
pros = pros.drop(['Rk','Tm','Lg','G','PA','AB','Pos\xa0Summary','GDP','SH'],axis = 1)
pros['OPS'] = pros['OPS▼']
pros = pros.drop(['OPS▼'],axis=1)
pros['Name'] = pros['Name'].str.replace("[*,#]", "") 
pros

,Name,Age,R,H,2B,3B,HR,RBI,SB,CS,...,SO,BA,OBP,SLG,OPS+,TB,HBP,SF,IBB,OPS
0,Christian Yelich,27.0,100.0,161.0,29.0,3.0,44.0,97.0,30.0,2.0,...,118.0,0.329,0.429,0.671,179.0,328.0,8.0,3.0,16.0,1.100
1,Mike Trout,27.0,110.0,137.0,27.0,2.0,45.0,104.0,11.0,2.0,...,120.0,0.291,0.438,0.645,185.0,303.0,16.0,4.0,14.0,1.083
2,Cody Bellinger,23.0,121.0,170.0,34.0,3.0,47.0,115.0,15.0,5.0,...,108.0,0.305,0.406,0.629,169.0,351.0,3.0,4.0,21.0,1.035
3,Nelson Cruz,38.0,81.0,141.0,26.0,0.0,41.0,108.0,0.0,1.0,...,131.0,0.311,0.392,0.639,166.0,290.0,7.0,3.0,8.0,1.031
4,Alex Bregman,25.0,122.0,164.0,37.0,2.0,41.0,112.0,5.0,1.0,...,83.0,0.296,0.423,0.592,162.0,328.0,9.0,8.0,2.0,1.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Khris Davis,31.0,61.0,106.0,11.0,0.0,23.0,73.0,0.0,0.0,...,146.0,0.220,0.293,0.387,82.0,186.0,3.0,2.0,3.0,0.679
132,Brandon Crawford,32.0,58.0,114.0,24.0,2.0,11.0,59.0,3.0,2.0,...,117.0,0.228,0.304,0.350,75.0,175.0,3.0,4.0,5.0,0.654
133,Yolmer Sanchez,27.0,59.0,125.0,20.0,4.0,2.0,43.0,5.0,4.0,...,117.0,0.252,0.318,0.321,73.0,159.0,5.0,3.0,1.0,0.638
134,Mallex Smith,26.0,70.0,116.0,19.0,9.0,6.0,37.0,46.0,9.0,...,141.0,0.227,0.300,0.335,74.0,171.0,11.0,1.0,0.0,0.635


In [5]:
names = []
for name in pros['Name']:
    names.append(name)
names = pd.DataFrame({'Pros':names})
minors_to_majors = pd.concat([names,data['Name']])
minors_to_majors['Pros'] = minors_to_majors['Pros'].str.replace("[\xa0]", " ")

In [6]:
match = minors_to_majors['Pros'].isin(minors_to_majors[0])
match = minors_to_majors[match]
matches = match['Pros'].head(50).dropna()
print(f'The amount of players that are in the minors dataframe that is now performing well in the majors is {matches.count()}')
print(matches)

The amount of players that are in the minors dataframe that is now performing well in the majors is 33
4               Alex Bregman
9                  Juan Soto
10               Pete Alonso
19               Jorge Soler
21             Rafael Devers
22               Jeff McNeil
23              Mookie Betts
24              Yoan Moncada
27               Kris Bryant
29              Trey Mancini
30                Matt Olson
39              Joc Pederson
41            Kyle Schwarber
47          Michael Conforto
48                Max Kepler
53              Matt Chapman
55               Javier Baez
58                 Luke Voit
63              Eloy Jimenez
65          Christian Walker
66             Franmil Reyes
69              Rhys Hoskins
71              Corey Seager
76             Marcell Ozuna
87          Daniel Vogelbach
88              Ryan McMahon
89         Andrew Benintendi
90     Vladimir Guerrero Jr.
93           Domingo Santana
96               Paul DeJong
105            Victor Roble

In [7]:
minor_to_major_df = df.loc[df['Name'].isin(matches)]

In [8]:
minor_attributes = df.describe()
call_ups = minor_to_major_df.describe()
minor_attributes = minor_attributes.drop(index='count')
call_ups = call_ups.drop(index='count')
diff = call_ups - minor_attributes
diff = diff.drop(['Rk','GDP','HBP','IBB','CS','SH','SF'],axis=1)
diff.drop(index=['min','max'])

,Age,OPS,G,PA,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,BA,OBP,SLG,TB
mean,-0.540696,0.018047,21.302957,97.880000,84.108435,16.282261,25.704696,5.720783,0.140087,6.578087,17.638174,0.784870,13.420696,15.397130,-0.002665,-0.001804,0.019899,51.439913
std,-1.091047,0.020114,4.775304,24.023789,18.275913,4.983934,5.748745,1.996755,0.002703,1.563034,4.265529,2.440633,7.088002,6.412176,0.003309,-0.003811,0.007803,9.409690
25%,1.000000,0.011500,17.000000,89.250000,90.750000,15.500000,22.250000,4.250000,0.000000,6.000000,20.500000,0.000000,11.250000,8.250000,0.000750,-0.002500,0.018500,50.500000
50%,0.000000,0.014000,26.500000,117.500000,109.000000,19.000000,28.500000,8.000000,0.000000,9.000000,21.000000,0.000000,11.000000,22.500000,-0.003500,0.000000,0.014000,65.000000
75%,-2.000000,0.013750,37.250000,164.500000,134.250000,20.750000,39.750000,8.750000,-1.000000,6.500000,21.250000,-1.000000,20.500000,24.750000,-0.003500,-0.000500,0.009000,63.750000


In [9]:
call_ups.drop(['Rk','GDP','HBP','IBB','CS'],axis=1)

,Age,OPS,G,PA,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,BA,OBP,SLG,TB,SH,SF
mean,21.891304,0.982065,87.586957,379.500000,327.630435,62.978261,104.108696,22.934783,3.326087,17.826087,64.652174,7.260870,44.108696,69.739130,0.322043,0.405196,0.576913,187.173913,0.434783,2.934783
std,2.213376,0.072828,33.222199,146.909156,125.366371,25.565158,38.992437,10.294340,2.732962,9.643200,27.182688,9.698648,25.270297,38.544309,0.032785,0.028379,0.054967,70.447239,1.003376,2.533352
min,17.000000,0.901000,30.000000,123.000000,108.000000,18.000000,35.000000,6.000000,0.000000,3.000000,11.000000,0.000000,11.000000,9.000000,0.250000,0.341000,0.505000,56.000000,0.000000,0.000000
25%,21.000000,0.935500,63.000000,279.250000,254.500000,46.500000,75.250000,15.250000,1.000000,11.000000,50.500000,1.000000,29.000000,38.250000,0.303750,0.382500,0.543500,140.500000,0.000000,1.000000
50%,22.000000,0.963500,85.000000,368.500000,323.000000,62.000000,98.500000,23.000000,3.000000,18.000000,62.000000,4.000000,38.000000,68.500000,0.322000,0.405000,0.568000,185.000000,0.000000,3.000000
75%,23.000000,1.002750,117.250000,503.750000,431.750000,79.750000,136.000000,30.750000,4.000000,21.750000,80.250000,8.000000,59.500000,93.750000,0.343500,0.427500,0.590000,232.750000,0.750000,4.750000
max,27.000000,1.218000,138.000000,634.000000,533.000000,118.000000,182.000000,50.000000,13.000000,43.000000,119.000000,45.000000,117.000000,162.000000,0.422000,0.483000,0.757000,325.000000,5.000000,10.000000


In [10]:
minor_attributes

,Rk,Age,OPS,G,PA,AB,R,H,2B,3B,...,SO,BA,OBP,SLG,TB,GDP,HBP,SH,SF,IBB
mean,50.500000,22.432000,0.964018,66.284000,281.620000,243.522000,46.696000,78.404000,17.214000,3.186000,...,54.342000,0.324708,0.40700,0.557014,135.734000,4.776000,4.490000,0.490000,2.430000,0.996000
std,28.894979,3.304423,0.052714,28.446894,122.885366,107.090458,20.581223,33.243692,8.297584,2.730259,...,32.132133,0.029477,0.03219,0.047164,61.037549,3.478221,3.914658,1.007934,1.962856,1.399966
min,1.000000,17.000000,0.900000,25.000000,113.000000,100.000000,13.000000,28.000000,2.000000,0.000000,...,5.000000,0.250000,0.32900,0.420000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.750000,20.000000,0.924000,46.000000,190.000000,163.750000,31.000000,53.000000,11.000000,1.000000,...,30.000000,0.303000,0.38500,0.525000,90.000000,2.000000,2.000000,0.000000,1.000000,0.000000
50%,50.500000,22.000000,0.949500,58.500000,251.000000,214.000000,43.000000,70.000000,15.000000,3.000000,...,46.000000,0.325500,0.40500,0.554000,120.000000,4.000000,4.000000,0.000000,2.000000,0.000000
75%,75.250000,25.000000,0.989000,80.000000,339.250000,297.500000,59.000000,96.250000,22.000000,5.000000,...,69.000000,0.347000,0.42800,0.581000,169.000000,7.000000,6.000000,1.000000,4.000000,1.000000
max,100.000000,33.000000,1.272000,138.000000,634.000000,533.000000,118.000000,182.000000,50.000000,18.000000,...,186.000000,0.422000,0.51300,0.774000,325.000000,19.000000,22.000000,7.000000,10.000000,8.000000


In [11]:
###Takeaway: 
'''The takeaways here are players that usually make it can be seen as playing more games at the minor league level, 
more plate appearances, hits, runs, RBI, total bases (homeruns and doubles), and walks. '''

'The takeaways here are players that usually make it can be seen as playing more games at the minor league level, \nmore plate appearances, hits, runs, RBI, total bases (homeruns and doubles), and walks. '

In [12]:
org = pd.read_csv('braves_milb_2019.csv')
org = org.dropna()
org['Name'] = org['Name'].str.replace("[*,#]", "") 
org.columns

Index(['Rk', 'Name', 'Age', 'Tm', 'Lev', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B',
       'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS▼', 'TB',
       'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Pos Summary'],
      dtype='object')

In [13]:
org = org.drop(['Rk','Tm','G','AB','SB','CS','SO','GDP','HBP','SH','SF','IBB','Pos\xa0Summary'],axis=1)
org = org.sort_values('TB',ascending=False)
org = org.set_index('Name')
org = org.head(30)
org.sort_values('OPS▼',ascending=False)

,Age,Lev,PA,R,H,2B,3B,HR,RBI,BB,BA,OBP,SLG,OPS▼,TB
Name,,,,,,,,,,,,,,,
Bryce Ball,20.0,"Rk,A",263.0,51.0,76.0,18.0,0.0,17.0,52.0,26.0,0.329,0.395,0.628,1.023,145.0
Adam Duvall,30.0,AAA,429.0,74.0,98.0,20.0,4.0,32.0,93.0,48.0,0.266,0.364,0.602,0.965,222.0
Austin Riley,22.0,"AAA,A",204.0,39.0,53.0,13.0,0.0,15.0,41.0,20.0,0.288,0.358,0.603,0.961,111.0
Travis Demeritte,24.0,AAA,399.0,68.0,97.0,28.0,2.0,20.0,73.0,51.0,0.286,0.387,0.558,0.944,189.0
Rafael Ortega,28.0,AAA,493.0,83.0,123.0,34.0,3.0,21.0,58.0,59.0,0.285,0.373,0.524,0.898,226.0
Trey Harris,23.0,"A,AA,A+",523.0,73.0,152.0,26.0,7.0,14.0,73.0,36.0,0.323,0.389,0.498,0.887,234.0
Ryan LaMarre,30.0,AAA,455.0,55.0,126.0,24.0,8.0,9.0,53.0,38.0,0.311,0.380,0.477,0.857,193.0
Alex Jackson,23.0,AAA,345.0,52.0,70.0,9.0,0.0,28.0,65.0,20.0,0.229,0.313,0.533,0.846,163.0
Drew Waters,20.0,"AA,AAA",573.0,80.0,163.0,40.0,9.0,7.0,52.0,39.0,0.309,0.360,0.459,0.819,242.0


In [14]:
avg_call_up_success = call_ups.drop(['Rk','G','AB','SB','CS','SO','GDP','HBP','SH','SF','IBB'],axis=1).iloc[0]
avg = pd.DataFrame(avg_call_up_success)
avg = avg.T
avg.index.names = ['Average']
avg.reset_index(drop=True, inplace=True)
avg

,Age,OPS,PA,R,H,2B,3B,HR,RBI,BB,BA,OBP,SLG,TB
0,21.891304,0.982065,379.5,62.978261,104.108696,22.934783,3.326087,17.826087,64.652174,44.108696,0.322043,0.405196,0.576913,187.173913


In [15]:
org[(avg['H'].values[0] <= org['H']) & (org['TB'] >= avg['TB'].values[0])]

,Age,Lev,PA,R,H,2B,3B,HR,RBI,BB,BA,OBP,SLG,OPS▼,TB
Name,,,,,,,,,,,,,,,
Drew Waters,20.0,"AA,AAA",573.0,80.0,163.0,40.0,9.0,7.0,52.0,39.0,0.309,0.360,0.459,0.819,242.0
Trey Harris,23.0,"A,AA,A+",523.0,73.0,152.0,26.0,7.0,14.0,73.0,36.0,0.323,0.389,0.498,0.887,234.0
Rafael Ortega,28.0,AAA,493.0,83.0,123.0,34.0,3.0,21.0,58.0,59.0,0.285,0.373,0.524,0.898,226.0
Cristian Pache,20.0,"AA,AAA",538.0,63.0,135.0,36.0,9.0,12.0,61.0,43.0,0.277,0.340,0.462,0.802,225.0
Andres Blanco,35.0,AAA,530.0,78.0,119.0,23.0,1.0,19.0,61.0,57.0,0.262,0.364,0.443,0.807,201.0
Ryan Casteel,28.0,AA,458.0,47.0,108.0,21.0,2.0,21.0,73.0,43.0,0.263,0.334,0.477,0.811,196.0
Ryan LaMarre,30.0,AAA,455.0,55.0,126.0,24.0,8.0,9.0,53.0,38.0,0.311,0.380,0.477,0.857,193.0
Greg Cullen,22.0,A,568.0,80.0,127.0,23.0,6.0,9.0,58.0,71.0,0.270,0.393,0.401,0.794,189.0


In [16]:
prospects = org[(avg['H'].values[0] <= org['H']) & (org['TB'] >= avg['TB'].values[0])| (org['HR'] >= avg['HR'].values[0])]
prospects.sort_values('TB',ascending=False)

,Age,Lev,PA,R,H,2B,3B,HR,RBI,BB,BA,OBP,SLG,OPS▼,TB
Name,,,,,,,,,,,,,,,
Drew Waters,20.0,"AA,AAA",573.0,80.0,163.0,40.0,9.0,7.0,52.0,39.0,0.309,0.360,0.459,0.819,242.0
Trey Harris,23.0,"A,AA,A+",523.0,73.0,152.0,26.0,7.0,14.0,73.0,36.0,0.323,0.389,0.498,0.887,234.0
Rafael Ortega,28.0,AAA,493.0,83.0,123.0,34.0,3.0,21.0,58.0,59.0,0.285,0.373,0.524,0.898,226.0
Cristian Pache,20.0,"AA,AAA",538.0,63.0,135.0,36.0,9.0,12.0,61.0,43.0,0.277,0.340,0.462,0.802,225.0
Adam Duvall,30.0,AAA,429.0,74.0,98.0,20.0,4.0,32.0,93.0,48.0,0.266,0.364,0.602,0.965,222.0
Andres Blanco,35.0,AAA,530.0,78.0,119.0,23.0,1.0,19.0,61.0,57.0,0.262,0.364,0.443,0.807,201.0
Ryan Casteel,28.0,AA,458.0,47.0,108.0,21.0,2.0,21.0,73.0,43.0,0.263,0.334,0.477,0.811,196.0
Ryan LaMarre,30.0,AAA,455.0,55.0,126.0,24.0,8.0,9.0,53.0,38.0,0.311,0.380,0.477,0.857,193.0
Greg Cullen,22.0,A,568.0,80.0,127.0,23.0,6.0,9.0,58.0,71.0,0.270,0.393,0.401,0.794,189.0


In [17]:
###dropping players that are on a different organization, in the pros, or just way older than the backtest avg age
prospects.drop(index=['Travis Demeritte','Ryan LaMarre','Adam Duvall','Andres Blanco'])

,Age,Lev,PA,R,H,2B,3B,HR,RBI,BB,BA,OBP,SLG,OPS▼,TB
Name,,,,,,,,,,,,,,,
Drew Waters,20.0,"AA,AAA",573.0,80.0,163.0,40.0,9.0,7.0,52.0,39.0,0.309,0.360,0.459,0.819,242.0
Trey Harris,23.0,"A,AA,A+",523.0,73.0,152.0,26.0,7.0,14.0,73.0,36.0,0.323,0.389,0.498,0.887,234.0
Rafael Ortega,28.0,AAA,493.0,83.0,123.0,34.0,3.0,21.0,58.0,59.0,0.285,0.373,0.524,0.898,226.0
Cristian Pache,20.0,"AA,AAA",538.0,63.0,135.0,36.0,9.0,12.0,61.0,43.0,0.277,0.340,0.462,0.802,225.0
Ryan Casteel,28.0,AA,458.0,47.0,108.0,21.0,2.0,21.0,73.0,43.0,0.263,0.334,0.477,0.811,196.0
Greg Cullen,22.0,A,568.0,80.0,127.0,23.0,6.0,9.0,58.0,71.0,0.270,0.393,0.401,0.794,189.0
Alex Jackson,23.0,AAA,345.0,52.0,70.0,9.0,0.0,28.0,65.0,20.0,0.229,0.313,0.533,0.846,163.0


In [18]:
###problems: code misses position breakdown and defensive factors (especially at catcher and infield) and 
###new college season pick-ups like catcher Shea Langeliers and SS Shewmake. 
#also biased toward outfielders (could look at positional averages of minors that now succeed in the pros)
#Position Breakdown: infielder: 1, outfielder: 4, catcher: 2
#top prospects based on backtesting and position summary: Alex Jackson, Drew Waters, Christian Pache, Greg Cullen
#missed: Contreras, Shewmake, Langeliers